In [0]:
%run "../includes/configuration"

In [0]:
v_source_file = dbutils.widgets.get("p_source_file")

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [0]:
qualifying_schema = StructType(fields=[StructField("qualifyId", IntegerType(), True),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), False),
                                    StructField("constructorId", IntegerType(), False),
                                    StructField("number", IntegerType(), False),
                                    StructField("position", IntegerType(), False),
                                    StructField("q1", StringType(), False),
                                    StructField("q2", StringType(), False),
                                    StructField("q3", StringType(), False)])

In [0]:
results_df = spark.read \
    .option("multiline", "true") \
    .schema(qualifying_schema) \
    .json(f"{raw_folder_path}/qualifying")

# results_df = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .json("/mnt/dldatabrickscoursedev001/raw/results.json")



In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
results_columnAlteration_df = results_df.withColumnRenamed("qualifyId", "qualifying_id") \
                                        .withColumnRenamed("raceId", "race_id") \
                                        .withColumnRenamed("driverId", "driver_id") \
                                        .withColumnRenamed("constructorId", "constructor_id") \
                                        .withColumn("data_source",lit(v_source_file))

In [0]:
%run "../functions/common_functions"

In [0]:
results_columnAlteration_df = add_ingest_date(results_columnAlteration_df)

In [0]:
results_columnAlteration_df.write \
    .mode("overwrite")  \
    .parquet(f"{processed_folder_path}/qualifying")

In [0]:
spark.read.parquet(f'{processed_folder_path}/qualifying').display()

In [0]:
dbutils.notebook.exit("success")